<a href="https://colab.research.google.com/github/sanghyun-ai/ktcloud_genai/blob/main/%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C/111_LLM_%ED%86%A0%ED%81%B0%EA%B3%BC%EC%9E%84%EB%B2%A0%EB%94%A9_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Token & Embedding 자세히 살펴보기**



---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---



# **Tokenization**

In [1]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [2]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

## LLM 토큰화 확인


**1. 모델 로드하고 토크나이저 지정하기**

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저를 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new version

**2.입력 프롬프트를 토큰으로 나누기**

In [4]:
prompt = '''Write an email apologizing to Sarah for the tragic gardening mishap.
Explain how it happened.<|assistant|>'''

# 입력 프롬프트를 토큰으로 나눕니다.
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# 텍스트를 생성합니다.
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20   # 토큰을 20개 생성
)

# 출력을 프린트합니다.
print(tokenizer.decode(generation_output[0]))  # decode 메서드: 토큰ID를 실제 텍스트로 변환

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Write an email apologizing to Sarah for the tragic gardening mishap.
Explain how it happened.<|assistant|> Subject: Sincere Apologies for the Gardening Mishap

Dear Sarah


In [5]:
print(input_ids)

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889,    13,  9544,  7420,
           920,   372,  9559, 29889, 32001]], device='cuda:0')


In [6]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.


Exp
lain
how
it
happened
.
<|assistant|>


In [7]:
generation_output

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889,    13,  9544,  7420,
           920,   372,  9559, 29889, 32001,  3323,   622, 29901,   317,  3742,
           406,  6225, 11763,   363,   278, 19906,   292,   341,   728,   481,
            13,    13, 29928,   799, 19235]], device='cuda:0')

In [8]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


## **토크나이저가 텍스트를 분할하는 방법**

1. 모델 설계시 모델 작성자가 토큰화 방법을 선택
    - GTP 모델 : BPE(byte pair encoding)
    - BERT 모델 : WordPiece
2. 토큰화 방법을 선택한 후에 어휘사전 크기와 특수 토큰 같은 토크나이저 설계상의 여러 가지 선택을 해야함
3. 토크나이저는 특정 데이터셋에서 훈련하여 해당 데이터셋을 표현하는 최상의 어휘사전을 구축해야 함

## **토큰 종류**

- **단어 토큰**
    - word2vec와 같은 초기 토큰화에 사용됨 --> 현재는 덜 사용됨
    - 지금도 추천 시스템과 같은 곳에서 사용
    - (단점: 훈련된 후에 데이터셋에 새롭게 추가된 단어는 처리할 수 없다.-->되도록 많은 어휘사전을 만들어야 한다.)

- **부분단어 토큰** (완전단어+부분완전단어 포함)
    - 새로운 단어를 (어휘사전에 포함되어 있을 가능성이 높은) 더 작은 단위로 나눈다.
    - **평균적으로 토큰당 세 개의 문자로 구성됨**
- **문자 토큰**
    - 대체할 원시 문자가 있기 때문에 새로운 단어를 잘 처리할 수 있다.
    - 토큰화는 쉽지만 모델링은 어렵다.--> 문자를 조합하는 정보까지 모델링해야함)
- **바이트 토큰**
    - 유니코드 문자를 표현하는 바이트로 토큰을 분할하는 방법
    - '토큰화-프리 인코딩' 라고 부름
    - 다국어 환경에서 경쟁력이 있다고 봄

## **훈련된 LLM 토큰나이저 비교하기**


- **중점 비교 요소**
    - **토큰화 방법** :  BEP, SentencePiece, WordPiece
    - **토크나이저 파라미터**
        - 어휘사전 크기 : 토크나이저가 어휘사전에 얼마나 많은 토큰을 포함할 건가?
        - 특수 토큰 : 모델이 추적해야할 특수 토큰은 무엇인가?
        - 대소문자 : 영어와 같은 대소문자를 어떻게 다뤄야 할까?

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        # 텍스트를 인코딩한 후 다시 디코딩했을 때 원본 텍스트와 동일해지려면
        # clean_up_tokenization_spaces를 False로 지정해야 합니다.
        # 현재 이 매개변수의 기본값은 None(True에 해당)이며
        # transformers 4.45에서 True로 바뀔 예정입니다.
        # https://github.com/huggingface/transformers/issues/31884
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t, clean_up_tokenization_spaces=False) +
            '\x1b[0m',
            end=' '
        )

In [10]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"		" four spaces:"    "
12.0*50=600
"""

- **BERT 베이스 모델(uncased)(2018)**
    - 토큰화 방법 : WordPiece
    - 어휘사전 크기 : 30, 522
    - 특수 토큰:
        - [UNK] unk_token : 토크나이저가 인코딩 방법을 모르는 토큰
        - [SEP] sep_toketn :  특정 작업에서 두 개의 텍스트를 구분하기 위한 토큰(cross-endcoder)
        - [PAD] pad_token :  모델 입력에서 사용되지 않는 위치를 채우기 위한 패딩 토큰 --> 모델은 특정 길이(문맥 크기)의 입력을 기대하기 때문
        - [CLS] cls_token :  분류 작업을 위한 특수 토큰
        - [MASK] mask_token : 훈련 과정 동안 일부 토큰을 감추기 위해 사용되는 마스킹 토큰
    - 특징 : 줄 바꿈 인코딩 정보를 알지 못함

In [11]:
show_tokens(text, "bert-base-uncased")  # Uncased : 대소문자를 구분하지 않음

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

- **BERT 베이스 모델(cased)(2018)**
    - 토큰화 방법 : WordPiece
    - 어휘사전 크기 : 28,996
    - 특수 토큰: uncased 버전과 동일

In [12]:
show_tokens(text, "bert-base-cased")

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

- **GPT-2(2019)**
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 50,257
    - 특수 토큰:
        - <|endoftext|>
    - 특징: 줄바꿈이 토크나이저 내에서 표현됨

In [13]:
show_tokens(text, "gpt2")


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"        " 
 12 . 0 * 50 = 600 
 

- **Flan-T5(2022)**
    - 토큰화 방법 : SentencePiece
    - 어휘사전 크기 : 32,100
    - 특수 토큰:
        - `<unk>` unk_token
        - `<pad>` pad_token
    - 특징 :
        - 줄바꿈이나 공백 토큰이 없음 --> 모델이 코드를 다루기 어렵다.
        - 이모자와 한자가 모두 <unk> 토큰으로 바꾸었음 --> 모델이 이런 토큰을 식별하지 못함

In [14]:
show_tokens(text, "google/flan-t5-small")

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " four spaces : " " 12. 0 * 50 = 600  </s> 

- **GPT-4(2023)**
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 100,000 이상
    - 특수 토큰:
        - `<endoftext>`
        - 중간 토큰을 채우도록 훈련됨. 세 개의 특수 토큰을 사용해 앞, 뒤에 나오는 텍스트를 고려해 LLM이 완성된 문장을 생성함
            - <|fim_prefix|>
            - <|fim_middle|>
            - <|fim_suffix|>
    - 특징 :
        - GPT-2 토크나이저와 비슷하게 동작
        - GPT-4는 4개의 공백을 하나의 토큰으로 표현
        - elif를 하나의 토큰으로 표현 --> 자연어외에 코드에 초점을 맞추고 있음
        - 더 적은 토큰을 사용해 대부분의 단어를 표현함(CAPITALIZATION 두 개의 토큰)

In [15]:
# 공식 토크나이저는 `tiktoken`이지만 허깅 페이스 플랫폼에 동일한 토크나이저가 있습니다.
show_tokens(text, "Xenova/gpt-4")


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      "
 12 . 0 * 50 = 600 
 

- **StarCoder2(2024)**
    - 코드 생성에 초점을 맞춘 150개의 파라미터를 가진 디코더 모델
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 49,152
    - 특수 토큰:
        - `<endoftext>`
        - 중간 채우기를 위한 토큰: `<fim_prefix>`, `<fim_middle>`, `<fim_suffix>`, `<fim_pad>`
        - `<filename>`, `<reponame>`, `<gh_stars>`
    - 특징 :
        - 코드를 표현할 때 문맥 관리가 중요(예를 들어 파일에서 다른 파일에 정의된 함수를 호출하는 경우)
        - 모델은 같은 저장소의 다른 파일에 있는 코드를 식별하고 다른 저장소에 있는 코드와 구분할 수 있어야함

In [16]:
show_tokens(text, "bigcode/starcoder2-15b")


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

- **Galatica**

    - 과학 지식에 초점을 맞추어 많은 과학 논문, 참고 자료, 지식 데이터에서 훈련됨.
    - 토큰화에 더 주의를 기울여 데이터셋에 있는 뉘앙스에 민감함 --> 인요, 추론, 수학, 펩타이드 서열, DNA 서열을 위한 특수 토큰이 있음
    -
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 50,000
    - 특수 토큰:
        - `<s>`
        - `<pad>`
        - `</s>`,
        - `<unk>`
        - 참조/인용은 [START_REF]와 [END_REF]로 감쌈
        - 단계별 추론: <work>는 모델이 CoT(chain-of-thought)추론에 사용하는 토큰
    - 특징 :
        - 코드를 염두에 둠, StarCoder2와 비슷하게 동작
        - 탭도 하나의 토큰으로 인코딩

In [17]:
show_tokens(text, "facebook/galactica-1.3b")


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : " 		 "  four  spaces : "      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

- **Phi-3(Llama 2)**
    - 여러 개의 특수 토큰을 추가한 Llama 2 토크나이저 재사용
    -
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 32,000
    - 특수 토큰:
        - `<|endoftext|>`
        - 채팅토큰
            - `<|user|>`,
            - `<|assistant|>`
            - `<|system|>`
    - 특징 :
        - 채팅에 초점을 맞춤

In [18]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :" 	 	 " four spaces :"    " 
 1 2 . 0 * 5 0 = 6 0 0 
 



---



## **한국어 특화된 토크나이저**

- **한글의 특수성**:
    - **교착어**: 조사가 붙어 단어 형태가 무한대로 변형 ("학교", "학교가", "학교에서", "학교로부터"...)
    - **띄어쓰기 불규칙**: SNS, 댓글 등에서 띄어쓰기가 일관되지 않음
    - **자모 조합**: 초성+중성+종성이 결합되어 하나의 음절 형성 (ex: ㄱ+ㅏ+ㅁ = 감)

- 발전 과정:
    - 2013-2016: KoNLPy, 형태소 분석기 (Mecab, Okt) 등장
    - 2018: 다국어 BERT에 한국어 포함되었으나 성능 부족
    - 2020: KoBERT (SKT) 출시 - 한국어 Wikipedia로 학습
    - 2021: KoGPT, KoBART 등 한국어 특화 모델 등장
    - 2022-2024: KoAlpaca, Polyglot-Ko, KULLM 등 다양한 한국어 LLM 개발

- 주요 참고자료:
    - SKT KoBERT: https://github.com/SKTBrain/KoBERT
    - Kakao KoGPT: https://github.com/kakaobrain/kogpt

### **[미션]** 한국어 LLM 찾아 토큰화 해보기.
- 허깅페이스에서 사전학습된 한국어 LLM을 여러 개(2개 이상) 찾고 위와 같이 토큰화 & 비교해보세요.

In [19]:
ko_text = """
    안녕하세요! 저는 인공지능을 공부하고 있습니다.,
    한글은 세종대왕이 창제하신 문자입니다. 😊,
    카카오와 네이버는 한국의 대표적인 IT 기업입니다.,
    자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하도록 하는 기술이다.,
"""

# 예제 : SKT의 KoBERT - 비교적 안정적
show_tokens(ko_text, "skt/kobert-base-v1")

 ᄋ [UNK] ᄒ [UNK] ᄋ [UNK] !  [UNK]  ᄋ ᅵ [UNK] ᄀ [UNK] ᅵ [UNK] ᄋ [UNK]  ᄀ [UNK] ᄒ [UNK] ᄀ [UNK]  ᄋ ᅵ [UNK] ᅵ [UNK] . ,  ᄒ [UNK] ᄀ [UNK] ᄋ [UNK]  [UNK] ᄋ [UNK] ᄋ ᅵ  [UNK] ᄒ [UNK] ᅵ [UNK]  [UNK] ᄋ ᅵ [UNK] ᅵ [UNK] .  [UNK] ,  ᄏ [UNK] ᄏ [UNK] ᄋ [UNK] ᄋ [UNK]  [UNK] ᄋ ᅵ [UNK]  ᄒ [UNK] ᄀ [UNK] ᄋ [UNK]  [UNK] ᄋ ᅵ [UNK] IT  ᄀ ᅵ ᄋ [UNK] ᄋ ᅵ [UNK] ᅵ [UNK] . ,  [UNK] ᄋ [UNK] ᄋ [UNK] ᅵ ( N L P ) [UNK]  ᄏ [UNK] ᄀ [UNK]  ᄋ ᅵ [UNK] ᄀ [UNK] ᄋ [UNK]  ᄋ [UNK] ᄋ [UNK]  ᄋ ᅵ ᄒ [UNK] ᄒ [UNK]  ᄒ [UNK]  ᄀ ᅵ [UNK] ᄋ ᅵ [UNK] . ,  [UNK] [UNK] 

In [20]:
# EleutherAI의 Polyglot-Ko 시리즈
show_tokens(ko_text, "EleutherAI/polyglot-ko-1.3b")  # Polyglot-Ko-1.3B
show_tokens(ko_text, "EleutherAI/polyglot-ko-5.8b")  # Polyglot-Ko-5.8B


        안녕 하 세요 !  저 는  인공지능 을  공부 하고  있 습니다 . , 
        한글 은  세종 대왕 이  창 제 하 신  문자 입니다 .   � � � � , 
        카카오 와  네이버 는  한국 의  대표 적 인  IT  기업 입니다 . , 
        자연 어 처리 ( NL P ) 는  컴퓨터 가  인간 의  언어 를  이해 하 도록  하 는  기술 이 다 . , 
 
        안녕 하 세요 !  저 는  인공지능 을  공부 하고  있 습니다 . , 
        한글 은  세종 대왕 이  창 제 하 신  문자 입니다 .   � � � � , 
        카카오 와  네이버 는  한국 의  대표 적 인  IT  기업 입니다 . , 
        자연 어 처리 ( NL P ) 는  컴퓨터 가  인간 의  언어 를  이해 하 도록  하 는  기술 이 다 . , 
 

In [21]:
# Beomi의 KcBERT - 댓글 데이터 특화
show_tokens(ko_text, "beomi/kcbert-base")

[CLS] 안녕 ##하세요 ! 저는 인공 ##지능 ##을 공부하고 있습니다 . , 한글 ##은 세종대왕 ##이 창 ##제 ##하신 문자 ##입니다 . 😊 , 카카오 ##와 네이버는 한국의 대표적인 IT 기업 ##입니다 . , 자연 ##어 ##처리 ( N ##L ##P ) 는 컴퓨터 ##가 인간의 언어 ##를 이해 ##하도록 하는 기술이 ##다 . , [SEP] 

In [22]:
# Hugging Face에서 보안 이슈 때문에 trust_remote_code=True 추가함
def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name,
                                              trust_remote_code=True)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t, clean_up_tokenization_spaces=False) +
            '\x1b[0m',
            end=' '
        )


# monologg의 KoBERT (Transformers 호환)
show_tokens(ko_text, "monologg/kobert")

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


[CLS] 안 녕 하세요 ! 저 는 인 공 지 능 을 공부 하고 있습니다 . , 한 글 은 세종 대 왕 이 창 제 하 신 문자 입니다 .  [UNK] , 카카오 와 네이버 는 한국의 대표적인 IT 기업 입니다 . , 자연 어 처리 ( N L P ) 는 컴퓨터 가 인간 의 언 어 를 이해 하도록 하는 기술 이다 . , [SEP] 



---



# **토큰 임베딩(Token Embedding)**

- **언어** --> **토큰의 시퀀스**
- **충분히 좋은 모델**을 **충분히 큰 토큰 집합**에서 훈련한다면 **훈련 데이터셋에 있는 복잡한 패턴을 포착**하기 시작한다.
- **Embedding** --> 수치표현, **언어에 있는 의미와 패턴을 포착하기 위한 수치 표현 공간**

- 토크나이저가 초기화되고 훈련되고 나면 이를 사용해 언어 모델을 훈련함
    - 사전 훈련된 언어 모델이 해당 토크나이저와 연결되는 이유
    - 모델을 재훈련하지 않고는 다른 토크나이저를 사용할 수 없음

### **[Quiz] "충분히 좋은 모델"은 어떤 것?**

문맥을 구분할 수 있는 모델

임베딩할 때 유사한 텍스트를 잘 구분해 낼 것

### **[Quiz] “충분히 큰 토큰 집합”의 조건은?**

- 중요한 단어/형태소 등이 과도하게 <unk> 로 표시되지 않을 것
- 너무 희귀하게 등장하는 단어까지 토큰에 포함하면 토큰 집합이 지나치게 커지므로 적당히 포함시킬 것




---



## (BERT와 같은)**언어 모델로 문맥을 고려한 단어 임베딩 만들기**

In [1]:
from transformers import AutoModel, AutoTokenizer

# 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# 언어 모델을 로드합니다.
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# 문장을 토큰으로 나눕니다.
tokens = tokenizer('Hello world', return_tensors='pt')

# 토큰을 처리합니다.
output = model(**tokens)[0]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [2]:
output.shape

# torch.Size([1, 4, 384])
#     1 : batch Size (한번에 처리하는 샘플(문장)개수)
#     4 : 토큰의 개수
#     384: 토큰의 임베딩 차원


torch.Size([1, 4, 384])

In [7]:
tokens

{'input_ids': tensor([[   1, 5365,  447,    2]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [3]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
Hello
world
[SEP]


In [4]:
output

tensor([[[-3.3060, -0.0507, -0.1098,  ..., -0.1704, -0.1618,  0.6932],
         [ 0.8918,  0.0740, -0.1583,  ...,  0.1869,  1.4760,  0.0751],
         [ 0.0871,  0.6364, -0.3050,  ...,  0.4729, -0.1829,  1.0157],
         [-3.1624, -0.1436, -0.0941,  ..., -0.0290, -0.1265,  0.7954]]],
       grad_fn=<NativeLayerNormBackward0>)

## **텍스트 임베딩** (문장과 전체 문서)

- 텍스트 임베딩이란 하나의 벡터로 토큰보다 긴 텍스트를 표현하는 것
- **텍스트 임베딩 모델**은 텍스트 조각을 입력받아 텍스트를 표현하고 유용한 형태로 의미를 포착하는 하나의 벡터를 만드는 모델

- sentence-transformers/all-mpnet-base-v2
    - 텍스트를 의미적으로 유사한 고품질의 벡터(임베딩)로 변환하는 데 특화된 문장 임베딩 모델
    - 문장 임베딩 분야에서 최고 수준의 성능을 보이는 모델 중 하나

In [5]:
from sentence_transformers import SentenceTransformer

# 모델을 로드합니다.
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 텍스트를 텍스트 임베딩으로 변환합니다.
vector = model.encode("Best movie ever!")

In [6]:
vector.shape

(768,)

### **예제 1: 영화 리뷰 감성 유사도 분석기**
텍스트 임베딩을 이용해 리뷰들의 의미적 유사도를 계산하고, 비슷한 감성을 가진 리뷰 찾기

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. 모델 로드
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 2. 영화 리뷰 데이터 (학생들이 직접 작성한 리뷰로 대체 가능)
reviews = [
    "이 영화는 정말 최고예요! 감동적이고 재미있었어요.",
    "인생 영화입니다. 눈물이 멈추지 않았어요.",
    "돈과 시간 낭비였어요. 지루하고 재미없었습니다.",
    "최악의 영화. 절대 추천하지 않습니다.",
    "배우들의 연기가 훌륭했고 스토리도 좋았어요.",
    "영화관에서 잠들었어요. 너무 지루했습니다."
]

# 3. 텍스트를 임베딩 벡터로 변환
embeddings = model.encode(reviews)

print(f"임베딩 벡터의 크기: {embeddings.shape}")  # (6, 768) - 6개 문장, 768차원
print(f"첫 번째 리뷰의 벡터 일부: {embeddings[0][:5]}")  # 첫 5개 값만 출력

# 4. 코사인 유사도 계산
similarity_matrix = cosine_similarity(embeddings)

# 5. 결과 시각화
print("\n=== 리뷰 간 유사도 매트릭스 ===")
print("(1.0에 가까울수록 유사, -1.0에 가까울수록 반대)")
print()

for i, review in enumerate(reviews):
    print(f"\n리뷰 {i+1}: {review[:30]}...")
    # 자기 자신을 제외하고 가장 유사한 리뷰 찾기
    similarities = similarity_matrix[i].copy()
    similarities[i] = -1  # 자기 자신 제외
    most_similar_idx = np.argmax(similarities)

    print(f"  → 가장 유사한 리뷰: 리뷰 {most_similar_idx+1}")
    print(f"     '{reviews[most_similar_idx][:40]}...'")
    print(f"  → 유사도 점수: {similarities[most_similar_idx]:.4f}")

# 6. 새로운 리뷰에 대해 가장 유사한 기존 리뷰 찾기
new_review = "정말 감동적인 영화였어요. 강력 추천합니다!"
new_embedding = model.encode([new_review])
new_similarities = cosine_similarity(new_embedding, embeddings)[0]

print(f"\n\n=== 새로운 리뷰 분석 ===")
print(f"새 리뷰: {new_review}")
most_similar = np.argmax(new_similarities)
print(f"가장 유사한 기존 리뷰 {most_similar+1}: {reviews[most_similar]}")
print(f"유사도: {new_similarities[most_similar]:.4f}")

임베딩 벡터의 크기: (6, 768)
첫 번째 리뷰의 벡터 일부: [ 0.02392089 -0.00769814 -0.00504546  0.02505269  0.0425802 ]

=== 리뷰 간 유사도 매트릭스 ===
(1.0에 가까울수록 유사, -1.0에 가까울수록 반대)


리뷰 1: 이 영화는 정말 최고예요! 감동적이고 재미있었어요....
  → 가장 유사한 리뷰: 리뷰 3
     '돈과 시간 낭비였어요. 지루하고 재미없었습니다....'
  → 유사도 점수: 0.8817

리뷰 2: 인생 영화입니다. 눈물이 멈추지 않았어요....
  → 가장 유사한 리뷰: 리뷰 4
     '최악의 영화. 절대 추천하지 않습니다....'
  → 유사도 점수: 0.9125

리뷰 3: 돈과 시간 낭비였어요. 지루하고 재미없었습니다....
  → 가장 유사한 리뷰: 리뷰 4
     '최악의 영화. 절대 추천하지 않습니다....'
  → 유사도 점수: 0.9079

리뷰 4: 최악의 영화. 절대 추천하지 않습니다....
  → 가장 유사한 리뷰: 리뷰 6
     '영화관에서 잠들었어요. 너무 지루했습니다....'
  → 유사도 점수: 0.9171

리뷰 5: 배우들의 연기가 훌륭했고 스토리도 좋았어요....
  → 가장 유사한 리뷰: 리뷰 3
     '돈과 시간 낭비였어요. 지루하고 재미없었습니다....'
  → 유사도 점수: 0.8902

리뷰 6: 영화관에서 잠들었어요. 너무 지루했습니다....
  → 가장 유사한 리뷰: 리뷰 4
     '최악의 영화. 절대 추천하지 않습니다....'
  → 유사도 점수: 0.9171


=== 새로운 리뷰 분석 ===
새 리뷰: 정말 감동적인 영화였어요. 강력 추천합니다!
가장 유사한 기존 리뷰 1: 이 영화는 정말 최고예요! 감동적이고 재미있었어요.
유사도: 0.9190


### **예제 2: 간단한 질문-답변 검색 시스템 (FAQ 봇)**
사용자 질문과 가장 유사한 FAQ를 찾아 답변을 제공

In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. 모델 로드
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 2. FAQ 데이터베이스 (대학교 컴퓨터공학과 FAQ 예시)
faq_database = {
    "questions": [
        "졸업 요건이 어떻게 되나요?",
        "전공 필수 과목은 무엇인가요?",
        "복수전공 신청은 어떻게 하나요?",
        "취업률이 어느 정도인가요?",
        "학점 인정은 어떻게 받나요?",
        "전과는 가능한가요?",
        "인턴십 프로그램이 있나요?",
        "기숙사 신청 방법을 알려주세요",
        "장학금 종류가 궁금합니다",
        "졸업 프로젝트는 필수인가요?"
    ],
    "answers": [
        "졸업하려면 전공 60학점, 교양 30학점 등 총 140학점이 필요하며, 평점 2.0 이상을 유지해야 합니다.",
        "자료구조, 알고리즘, 운영체제, 데이터베이스, 컴퓨터구조가 전공 필수 과목입니다.",
        "복수전공은 2학년 2학기부터 신청 가능하며, 학사포털에서 온라인으로 신청하시면 됩니다.",
        "최근 3년간 평균 취업률은 92%이며, 대기업 및 IT 기업 취업률이 높습니다.",
        "타 대학 학점 인정은 학점교류 협정 대학에 한하며, 학과 사무실에 신청서를 제출하시면 됩니다.",
        "전과는 1학년 말에 가능하며, 성적 및 TO에 따라 선발됩니다.",
        "여름/겨울 방학 중 산학협력 인턴십 프로그램을 운영하고 있으며, 학점으로 인정됩니다.",
        "기숙사는 매 학기 초 학생포털에서 신청하며, 거리와 성적 순으로 선발됩니다.",
        "성적우수 장학금, 국가장학금, 근로장학금 등 다양한 장학 제도가 있습니다.",
        "네, 졸업 프로젝트는 필수이며 4학년 1, 2학기에 걸쳐 진행됩니다."
    ]
}

# 3. FAQ 질문들을 임베딩으로 변환 (미리 계산해두면 효율적)
print("\nFAQ 데이터베이스를 임베딩으로 변환 중...")
faq_embeddings = model.encode(faq_database["questions"])
print(f"총 {len(faq_database['questions'])}개의 FAQ가 준비되었습니다.")
print("-" * 70,'\n')


# 4. 사용자 질문 처리 함수
def find_answer(user_question, top_k=3):
    """
    사용자 질문에 가장 유사한 FAQ를 찾아 답변을 반환

    Parameters:
        user_question: 사용자의 질문 (문자열)
        top_k: 상위 몇 개의 유사한 질문을 보여줄지
    """
    # 사용자 질문을 임베딩으로 변환
    question_embedding = model.encode([user_question])

    # 유사도 계산
    similarities = cosine_similarity(question_embedding, faq_embeddings)[0]

    # 상위 k개의 가장 유사한 질문 찾기
    top_indices = np.argsort(similarities)[-top_k:][::-1]

    print(f"✅ 질문: {user_question}")
    print("=" * 70)

    for rank, idx in enumerate(top_indices, 1):
        print(f"\n\t[{rank}순위] 유사도: {similarities[idx]:.4f}")
        print(f"\t유사한 질문: {faq_database['questions'][idx]}")
        print(f"\t답변: {faq_database['answers'][idx]}")
        print("\t", "-" * 70)

    # 가장 유사한 답변 반환
    best_match_idx = top_indices[0]
    return faq_database['answers'][best_match_idx], similarities[best_match_idx]


# 5. 테스트 시나리오
test_questions = [
    "졸업하려면 학점을 얼마나 들어야 해요?",  # '졸업 요건'과 유사
    "인턴 할 수 있나요?",  # '인턴십 프로그램'과 유사
    "다른 과로 옮길 수 있어요?",  # '전과'와 유사
    "AI 수업을 듣고 싶어요"  # FAQ에 없는 질문
]


for test_q in test_questions:
    answer, similarity = find_answer(test_q, top_k=2)
    print(f"\n\t{'='*70}")
    print(f"\t★ 최종 답변 (신뢰도: {similarity:.4f})")

    if similarity < 0.5:  # 유사도가 낮으면 경고
        print("⚠️ 유사도가 낮습니다. 관련된 질문이 FAQ에 없을 수 있습니다.")

    print(f"\t답변: {answer}")
    print(f"\t{'='*70}\n\n")


FAQ 데이터베이스를 임베딩으로 변환 중...
총 10개의 FAQ가 준비되었습니다.
---------------------------------------------------------------------- 

✅ 질문: 졸업하려면 학점을 얼마나 들어야 해요?

	[1순위] 유사도: 0.8883
	유사한 질문: 복수전공 신청은 어떻게 하나요?
	답변: 복수전공은 2학년 2학기부터 신청 가능하며, 학사포털에서 온라인으로 신청하시면 됩니다.
	 ----------------------------------------------------------------------

	[2순위] 유사도: 0.8542
	유사한 질문: 학점 인정은 어떻게 받나요?
	답변: 타 대학 학점 인정은 학점교류 협정 대학에 한하며, 학과 사무실에 신청서를 제출하시면 됩니다.
	 ----------------------------------------------------------------------

	★ 최종 답변 (신뢰도: 0.8883)
	답변: 복수전공은 2학년 2학기부터 신청 가능하며, 학사포털에서 온라인으로 신청하시면 됩니다.


✅ 질문: 인턴 할 수 있나요?

	[1순위] 유사도: 0.8241
	유사한 질문: 복수전공 신청은 어떻게 하나요?
	답변: 복수전공은 2학년 2학기부터 신청 가능하며, 학사포털에서 온라인으로 신청하시면 됩니다.
	 ----------------------------------------------------------------------

	[2순위] 유사도: 0.7904
	유사한 질문: 학점 인정은 어떻게 받나요?
	답변: 타 대학 학점 인정은 학점교류 협정 대학에 한하며, 학과 사무실에 신청서를 제출하시면 됩니다.
	 ----------------------------------------------------------------------

	★ 최종 답변 (신뢰도: 0.8241)
	답변: 복수전공은 2학년 2학기부터 신청 가능하며,

### **[미션] 나만의 FAQ 봇 만들기**
앞에서 실습해 본 `예제 2: 간단한 질문-답변 검색 시스템 (FAQ 봇)`에 자신만의 적절한 데이터를 수집하고 반영하여 OOO 봇을 만들어 보세요.

In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. 모델 로드
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2') # 기존 sentence-transformers/all-mpnet-base-v2

# 2. FAQ 데이터베이스 (메이플스토리 '렌' 관련)
faq_database = {
    "questions": [
        "렌(Len)은 어떤 캐릭터인가요?",
        "렌(Len)은 언제 업데이트된 캐릭터인가요?",
        "렌은 어떤 직업군(모험가/영웅/아니마 등)에 속하나요?",
        "렌의 무기 종류와 보조 무기는 무엇인가요?",
        "렌의 장비 특징은 무엇인가요?"
        "렌의 콘셉트(스토리 배경)는 어떤 내용인가요?",
        "렌의 주요 스킬 구조와 전투 스타일은 어떤가요?",
        "렌은 전투할 때 조작 난이도가 높은 편인가요?",
        "렌은 사냥과 보스전 중 어디에 더 강점이 있나요?",
        "렌의 링크 스킬 효과는 무엇인가요?",
        "렌을 처음 키울 때 추천하는 스탯 배분은 무엇인가요?",
        "렌은 어떤 능력치를 주로 올리면 되나요?",
        "렌의 초보자 난이도는 어느 정도인가요?",
        "렌은 다른 직업보다 배우기 쉬운 편인가요?",
        "렌의 단점이나 주의해야 할 점은 무엇인가요?",
        "렌과 시너지가 좋은 다른 직업은 무엇인가요?"
    ],
    "answers": [
        "렌은 아니마 직업군에 속하는 전사 캐릭터로, 동양풍의 무기를 다루는 독특한 콘셉트를 가지고 있습니다. 두 종류의 검을 활용하며, 스토리와 전투 스타일 모두 신선하다는 평가를 받고 있습니다.",
        "렌은 2025년 여름 대규모 패치인 'Assemble!' 업데이트에서 처음 공개된 신규 직업입니다. 약 3년 만의 신캐릭터라 많은 유저들의 기대와 관심을 모았습니다.",
        "렌은 아니마(Anima) 계열에 속하는 전사형 캐릭터입니다. 아니마 직업군 특유의 동양적 세계관을 반영하면서도, 전투 스타일은 비교적 간단하고 직관적으로 설계되었습니다.",
        "렌의 주 무기는 한손검이며, 'Plum Blossom Sword'와 'Lost Soul Sword'라는 두 가지 검을 상황에 맞게 사용합니다. 보조 무기는 없습니다.",
        "렌은 보조 무기 대신 두 검을 번갈아 사용한다는 특징이 있습니다. 이 때문에 장비 세팅은 일반 전사와 유사하지만, 무기와 스킬 연계 방식이 조금 더 독창적입니다.",
        "렌은 이무기 ‘사야(Saya)’와 함께 모험을 떠나는 전사로, 동양적인 색채가 짙은 세계관을 배경으로 합니다. 스토리 자체가 캐릭터의 테마와 스킬 연계에도 반영되어 있어 몰입감을 높입니다.",
        "렌의 스킬은 홀드(hold) 방식으로 키를 누르고 유지하며 연속 공격을 이어가는 구조입니다. 또 자동 발동 스킬이 많아 초보자도 쉽게 다룰 수 있으며, 전투 중 이동과 점프가 가능해 유연한 스타일을 보여줍니다.",
        "렌은 직관적인 스킬 구조 덕분에 조작 난이도가 낮은 편으로 평가됩니다. 기본 공격이 단순하면서도 연계가 매끄럽기 때문에 숙련도가 낮은 유저도 쉽게 다룰 수 있습니다.",
        "렌은 사냥과 보스전 모두 무난한 성능을 보여주는 밸런스형 캐릭터입니다. 특히 보스전에서는 링크 스킬을 통해 생존력이 강화되어 안정적으로 싸울 수 있다는 장점이 있습니다.",
        "렌의 링크 스킬은 퍼센트 HP 기반 공격 피해를 감소시키는 효과를 가지고 있습니다. 이 덕분에 강력한 보스의 공격을 견디는 데 유리하며, 파티 플레이에서도 큰 도움이 됩니다.",
        "렌은 STR(힘)을 주 스탯으로 삼기 때문에 자동 스탯 배분을 사용하는 것이 가장 효율적입니다. 별도의 수동 분배가 필요하지 않아 초보자도 부담 없이 성장시킬 수 있습니다.",
        "렌은 전사형 캐릭터로서 공격력과 관련된 STR을 집중적으로 올리는 것이 핵심입니다. 자동 분배를 사용하면 자연스럽게 STR 위주로 배분되므로 안정적입니다.",
        "렌은 초보자도 쉽게 다룰 수 있는 난이도 중하 수준으로 평가됩니다. 조작이 단순하고 자동 발동 스킬이 많아, 처음 시작하는 유저도 빠르게 적응할 수 있습니다.",
        "네, 렌은 다른 직업군에 비해 조작 난이도가 낮아 배우기 쉬운 편입니다. 따라서 메이플스토리를 처음 접하는 유저나 서브 캐릭터로 가볍게 키우기에도 적합합니다.",
        "렌은 지속 딜링 구조가 최적화되지 않아 숙련도에 따라 성능 차이가 발생할 수 있습니다. 또한 신규 직업 특성상 추후 밸런스 패치가 이루어질 가능성이 높다는 점도 염두에 두어야 합니다.",
        "렌은 방어적인 성격을 가진 데몬어벤져나 팔라딘과 시너지가 잘 맞습니다. 또한 경험치 증가 링크 스킬을 가진 메르세데스 등과 함께하면 육성과 파티 플레이에서 더욱 강력한 효과를 발휘할 수 있습니다."
    ]
}

# 3. FAQ 질문들을 임베딩으로 변환 (미리 계산해두면 효율적)
print("\nFAQ 데이터베이스를 임베딩으로 변환 중...")
faq_embeddings = model.encode(faq_database["questions"])
print(f"총 {len(faq_database['questions'])}개의 FAQ가 준비되었습니다.")
print("-" * 70,'\n')


# 4. 사용자 질문 처리 함수
def find_answer(user_question, top_k=3):
    """
    사용자 질문에 가장 유사한 FAQ를 찾아 답변을 반환

    Parameters:
        user_question: 사용자의 질문 (문자열)
        top_k: 상위 몇 개의 유사한 질문을 보여줄지
    """
    # 사용자 질문을 임베딩으로 변환
    question_embedding = model.encode([user_question])

    # 유사도 계산
    similarities = cosine_similarity(question_embedding, faq_embeddings)[0]

    # 상위 k개의 가장 유사한 질문 찾기
    top_indices = np.argsort(similarities)[-top_k:][::-1]

    print(f"✅ 질문: {user_question}")
    print("=" * 70)

    for rank, idx in enumerate(top_indices, 1):
        print(f"\n\t[{rank}순위] 유사도: {similarities[idx]:.4f}")
        print(f"\t유사한 질문: {faq_database['questions'][idx]}")
        print(f"\t답변: {faq_database['answers'][idx]}")
        print("\t", "-" * 70)

    # 가장 유사한 답변 반환
    best_match_idx = top_indices[0]
    return faq_database['answers'][best_match_idx], similarities[best_match_idx]


# 5. 테스트 시나리오
test_questions = [
    "렌은 어떤 계열 직업인가요?",  # '직업군'과 유사
    "렌과 같이 키우기 좋은 캐릭터는 뭐야?",  # '직업 시너지'와 유사
    "렌 스탯은 어떻게 올리면 좋을까?",  # '스탯 배분'와 유사
    "렌의 200렙에서 추천하는 사냥터를 추천해줘"  # FAQ에 없는 질문
]


for test_q in test_questions:
    answer, similarity = find_answer(test_q, top_k=2)
    print(f"\n\t{'='*70}")
    print(f"\t★ 최종 답변 (신뢰도: {similarity:.4f})")

    if similarity < 0.5:  # 유사도가 낮으면 경고
        print("⚠️ 유사도가 낮습니다. 관련된 질문이 FAQ에 없을 수 있습니다.")

    print(f"\t답변: {answer}")
    print(f"\t{'='*70}\n\n")



FAQ 데이터베이스를 임베딩으로 변환 중...
총 15개의 FAQ가 준비되었습니다.
---------------------------------------------------------------------- 

✅ 질문: 렌은 어떤 계열 직업인가요?

	[1순위] 유사도: 0.8372
	유사한 질문: 렌은 어떤 직업군(모험가/영웅/아니마 등)에 속하나요?
	답변: 렌은 아니마(Anima) 계열에 속하는 전사형 캐릭터입니다. 아니마 직업군 특유의 동양적 세계관을 반영하면서도, 전투 스타일은 비교적 간단하고 직관적으로 설계되었습니다.
	 ----------------------------------------------------------------------

	[2순위] 유사도: 0.7343
	유사한 질문: 렌과 시너지가 좋은 다른 직업은 무엇인가요?
	답변: 렌은 지속 딜링 구조가 최적화되지 않아 숙련도에 따라 성능 차이가 발생할 수 있습니다. 또한 신규 직업 특성상 추후 밸런스 패치가 이루어질 가능성이 높다는 점도 염두에 두어야 합니다.
	 ----------------------------------------------------------------------

	★ 최종 답변 (신뢰도: 0.8372)
	답변: 렌은 아니마(Anima) 계열에 속하는 전사형 캐릭터입니다. 아니마 직업군 특유의 동양적 세계관을 반영하면서도, 전투 스타일은 비교적 간단하고 직관적으로 설계되었습니다.


✅ 질문: 렌과 같이 키우기 좋은 캐릭터는 뭐야?

	[1순위] 유사도: 0.8390
	유사한 질문: 렌(Len)은 어떤 캐릭터인가요?
	답변: 렌은 아니마 직업군에 속하는 전사 캐릭터로, 동양풍의 무기를 다루는 독특한 콘셉트를 가지고 있습니다. 두 종류의 검을 활용하며, 스토리와 전투 스타일 모두 신선하다는 평가를 받고 있습니다.
	 ----------------------------------------------------------------------

	[

## **임베딩으로 노래 추천하기**

- 데이터셋: 코넬대학교 슈오첸이 모은 데이터셋(미국 전역에 있는 수백개의 라이도 방송국에서 가져온 재생목록)
    - 노래 재생목록 :
        - https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt
    - 노래 메타데이터: (제목, 아티스트)
        - https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt


In [31]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 119.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 15.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
open

- 데이터셋 로드하기

In [32]:
import pandas as pd
from urllib import request

# 재생목록 데이터셋 파일을 가져옵니다.
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# 재생목록 파일을 파싱합니다. 처음 두 줄은 메타데이터만 담고 있으므로 건너뜁니다.
lines = data.read().decode("utf-8").split('\n')[2:]
print(len(lines))

# 하나의 노래만 있는 재생목록은 삭제합니다.
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# 노래의 메타데이터를 로드합니다.
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

11138


In [33]:
print('재생목록 #1:\n ', playlists[0], '\n')
print('재생목록 #2:\n ', playlists[1])

재생목록 #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

재생목록 #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117', '118', 

- Word2Vec 훈련하기
    - 결과물: 각 노래에 대해 계산된 임베딩 결과 --> 이 임베딩으로 비슷한 노래를 찾을 수 있다.)

In [34]:
from gensim.models import Word2Vec

# Word2Vec 모델을 훈련합니다.
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
song_id = 2172

# 노래 ID 2172와 비슷한 노래를 찾으라고 모델에게 요청합니다.
model.wv.most_similar(positive=str(song_id))

In [ ]:
print(songs_df.iloc[2172])

In [ ]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# 추천 노래 출력
print_recommendations(2172)

In [ ]:
print_recommendations(842)